# Raport
Autorzy: Staroń Kacper, Szypuła Jakub, Urbala Anna

## Zbiór danych
Wybraliśmy zbiór [cervical cancer](https://datahub.io/machine-learning/cervical-cancer) zawierający dane medyczne pacjentek, u części z nich zdiagnozowany został rak szyjki macicy. Całość składa się 835 rekordów. Kolumny:
- Age - wiek
- Number of sexual partners - liczba partnerów seksualnych
- First sexual intercourse - wiek inicjacji seksualnej
- Num of pregnancies - liczba ciąż
- Smokes - czy pali
- Smokes (years) - od ilu lat pali
- Smokes (packs/year) - liczba wypalanych paczek na rok
- Hormonal Contraceptives - czy stosuje antykoncepcję hormonalną
- Hormonal Contraceptives (years) - od ilu lat stosuje antykoncepcję hormonalną
- IUD - czy stosuje wkładkę domaciczną
- IUD (years) - od ilu lat stosuje wkładkę domaciczną
- STDs - czy ma/miała chorobę przenoszą drogą płciową
- STDs (number) - ile ma/miała chorób przenoszonych drogą płciową (kolejne kolumny zaczynające się od STDs oznaczają konkretne choroby)
- STDs:condylomatosis
- STDs:cervical condylomatosis
- STDs:vaginal condylomatosis
- STDs:vulvo-perineal condylomatosis
- STDs:syphilis
- STDs:pelvic inflammatory disease
- STDs:genital herpes
- STDs:molluscum contagiosum
- STDs:AIDS
- STDs:HIV
- STDs:Hepatitis B
- STDs:HPV
- STDs: Number of diagnosis
- STDs: Time since first diagnosis
- STDs: Time since last diagnosis
- Dx:Cancer - czy miała zdiagnozowanego raka
- Dx:CIN - czy miała zdiagnozowanego raka śródnabłonkowego
- Dx:HPV - czy były zakażona HPV
- Dx - czy była zdiagnozowane któreś z powyższych
Kolumny zmiennych celu (metoda diagnozy raka szyjki macicy):
- Hinselmann
- Schiller
- Citology
- Biopsy

Większość z tych kolumn zawiera braki danych ze względów osobistych pacjentek. 
### Zmienna celu
Celem zadania jest prognozowanie raka szyjki macicy na podstawie posiadanych danych.

W zbiorze wyjściowym są cztery kolumny celu, jednak postanowiliśmy ograniczyć się do jednej. Stworzyliśmy własną kolumnę celu, która jest sumą logiczną tamtych czterech. Nazwaliśmy ją *cancer*.

## Analiza zbioru

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport

data = pd.read_csv('cervical-cancer_csv.csv')

profile = ProfileReport(data, title='Pandas Profiling Report', html={'style':{'full_width':True}})
profile.to_notebook_iframe()

ModuleNotFoundError: No module named 'pandas_profiling'

In [ ]:
# Rozkład zmiennych kategorycznych
categorical = ['Smokes', 'Hormonal Contraceptives', 'IUD', 'STDs', 'STDs (number)', 'STDs:condylomatosis', 
               'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis', 'STDs:vulvo-perineal condylomatosis',
              'STDs:syphilis', 'STDs:pelvic inflammatory disease', 'STDs:genital herpes', 'STDs:molluscum contagiosum',
              'STDs:AIDS', 'STDs:HIV', 'STDs:Hepatitis B', 'STDs:HPV', 'STDs: Number of diagnosis',
               'Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller', 'Citology', 'Biopsy']

fig, axes = plt.subplots(nrows=13, ncols=2, figsize=(12, 36))
axes = axes.flatten()
total = len(data)
for idx, ax in enumerate(axes):
    plt.sca(ax)
    ax = sns.countplot(x = categorical[idx], data=data)
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:3.0f}%'.format(100 * height/total),
            ha="center")
sns.despine()
plt.tight_layout()
plt.show()

### Problem braku danych
Ze względu na liczne braki danych podjęliśmy następujące decyzje:
- ze względu na (prawie) brak danych innych niż 0 usuwamy następujące kolumny:
    * STDs:cervical condylomatosis
    * STDs:vaginal condylomatosis
    * STDs:pelvic inflammatory disease
    * STDs:genital herpes
    * STDs:molluscum contagiosum
    * STDs:AIDS
    * STDs:Hepatitis B
    * STDs:HPV
    * Dx:CIN
- pozostałe kolumny kategoryczne: w związku z tym, że użyjemy OHE, tworzymy dodatkową kolumnę na brak danych
- dane ciągłe: będą normalizowane, więc ustawienie braków na 0 (czyli średnia) jest bezpieczne

### Wstępny szablon imputacji i obróbki danych

In [ ]:
from sklearn.preprocessing import StandardScaler
dane = pd.read_csv('cervical-cancer_csv.csv')

# usuwanie kolumn
dane = dane.drop(['STDs:cervical condylomatosis', 
         'STDs:vaginal condylomatosis', 
         'STDs:pelvic inflammatory disease', 
         'STDs:genital herpes',
         'STDs:molluscum contagiosum',
         'STDs:AIDS',
         'STDs:Hepatitis B',
         'STDs:HPV', 'Dx:CIN'], axis=1)

# uzupełnianie braków i kodowanie zmiennych kategorycznych
def column_nodata(df, column_name):
    df[column_name + "_null"] = df[column_name].apply(lambda x: 1 if pd.isnull(x) else 0)
    df[column_name] = df[column_name].fillna(0)

def replace_in_column(df, column_name, src, dst):
    df[column_name] = df[column_name].replace(to_replace=src, value=dst)

replace_in_column(dane, 'STDs (number)', [3, 4], 2)
replace_in_column(dane, 'STDs: Number of diagnosis', [2,3], 1)

nodata_categories = [
    'Smokes',
    'Hormonal Contraceptives',
    'IUD',
    'STDs',
    'STDs (number)',
    'STDs:condylomatosis',
    'STDs:vulvo-perineal condylomatosis',
    'STDs:syphilis',
    'STDs:HIV'
]

for category in nodata_categories:
    column_nodata(dane, category)
    
dane = pd.concat([dane, pd.get_dummies(dane['STDs (number)'], prefix='STDs_')],axis=1)
dane.drop(['STDs (number)'],axis=1, inplace=True)

# standaryzacja
numerical = ['Age', 'Number of sexual partners', 'First sexual intercourse', 'Num of pregnancies', 'Smokes (years)', 
            'Smokes (packs/year)', 'Hormonal Contraceptives (years)', 'IUD (years)', 'STDs: Time since first diagnosis',
            'STDs: Time since last diagnosis']

scaler = StandardScaler() 
dane_scaled = scaler.fit_transform(dane[numerical])
d2 = pd.DataFrame(dane_scaled, columns = numerical)
dane[numerical] = d2[numerical]

# stworzenie jednego targetu
targets = ['Hinselmann', 'Schiller', 'Citology', 'Biopsy']
def has_cancer(row):
    for target in targets:
        if row[target] == 1:
            return 1
    return 0

dane['cancer'] = dane.apply(lambda row: has_cancer(row), axis=1)
dane = dane.drop(targets, axis=1)

## Oświadczenie
Potwierdzam samodzielność powyższej pracy oraz niekorzystanie przeze mnie z niedozwolonych źródeł